# Setup

In [1]:
import pandas as pd
from google.cloud import securitycenter_v1

scc_client = securitycenter_v1.SecurityCenterClient()

# Functions

In [2]:
def partition(l, n):
    for i in range(0, len(l), n):
        yield l[i : i + n]

In [3]:
def get_findings(org_id):
    yield from scc_client.list_findings(
        securitycenter_v1.ListFindingsRequest(
            parent=f"organizations/{org_id}/sources/-",
            page_size=1000,
        )
    )

In [4]:
def get_assets(org_id, findings):
    resource_names = {f.finding.resource_name for f in findings}
    filters = [
        f'securityCenterProperties.resourceName="{resource_name}"'
        for resource_name in resource_names
    ]

    for subfilter in partition(filters, 1000):
        yield from scc_client.list_assets(
            securitycenter_v1.ListAssetsRequest(
                parent=f"organizations/{org_id}",
                filter=" OR ".join(subfilter),
                page_size=1000,
            )
        )

In [5]:
def join_findings_assets(findings, assets):
    fdf = pd.DataFrame(
        {
            "category": f.finding.category,
            "scanner_name": f.finding.source_properties.get("ScannerName"),
            "project_id": f.resource.project_display_name,
            "event_time": f.finding.event_time,
            "create_time": f.finding.create_time,
            "resource_name": f.finding.resource_name,
            "name": f.finding.name,
            "severity": f.finding.severity.name,
        }
        for f in findings
    )

    adf = pd.DataFrame(
        {
            "resource_name": a.asset.security_center_properties.resource_name,
            "resource_display_name": a.asset.security_center_properties.resource_display_name,
        }
        for a in assets
    )

    return fdf.merge(adf, on="resource_name", how="left")

In [6]:
def get_joined_findings_assets(org_id):
    findings = list(get_findings(org_id))
    assets = list(get_assets(org_id, findings))
    return join_findings_assets(findings, assets)

# Exploration

## Load Findings and Assets

In [7]:
findings = list(get_findings("129624834409"))
assets = list(get_assets("129624834409", findings))

## Example Finding

In [8]:
f = findings[20000]
f

finding {
  name: "organizations/129624834409/sources/14464885714567633669/findings/8a297d34b5e44daad89338a53f45cc60"
  parent: "organizations/129624834409/sources/14464885714567633669"
  resource_name: "//compute.googleapis.com/projects/csam-green-test-project-0/global/firewalls/4325101218256444732"
  state: ACTIVE
  category: "OPEN_FIREWALL"
  external_uri: "https://console.cloud.google.com/networking/firewalls/details/test-numbered-proto-29?project=csam-green-test-project-0"
  source_properties {
    key: "ActivationTrigger"
    value {
      string_value: "Allows all IP addresses"
    }
  }
  source_properties {
    key: "AllowedIpRange"
    value {
      string_value: "All"
    }
  }
  source_properties {
    key: "ExceptionInstructions"
    value {
      string_value: "Add the security mark \"allow_open_firewall\" to the asset with a value of \"true\" to prevent this finding from being activated again."
    }
  }
  source_properties {
    key: "Explanation"
    value {
      stri

## Example Asset

In [9]:
a = assets[0]
a

asset {
  name: "organizations/129624834409/assets/2660417069574448459"
  security_center_properties {
    resource_name: "//cloudresourcemanager.googleapis.com/projects/137966506042"
    resource_type: "google.cloud.resourcemanager.Project"
    resource_parent: "//cloudresourcemanager.googleapis.com/folders/175430757873"
    resource_project: "//cloudresourcemanager.googleapis.com/projects/137966506042"
    resource_owners: "user:clingeric@google.com"
    resource_display_name: "bq4g-teams"
    resource_parent_display_name: "teams"
    resource_project_display_name: "bq4g-teams"
  }
  resource_properties {
    key: "createTime"
    value {
      string_value: "2020-12-17T18:28:21.445Z"
    }
  }
  resource_properties {
    key: "lifecycleState"
    value {
      string_value: "ACTIVE"
    }
  }
  resource_properties {
    key: "name"
    value {
      string_value: "bq4g-teams"
    }
  }
  resource_properties {
    key: "parent"
    value {
      string_value: "{\"id\":\"175430757873\

## Joined

In [11]:
df = get_joined_findings_assets("129624834409")
df

,category,scanner_name,project_id,event_time,create_time,resource_name,name,severity,resource_display_name
0,Persistence: New User Agent,None,,2021-03-03 23:38:35.510000+00:00,2021-03-03 23:38:36.401000+00:00,//compute.googleapis.com/projects/jinmmin-sand...,organizations/129624834409/sources/13627762962...,HIGH,NaN
1,Persistence: New User Agent,None,,2021-03-03 23:30:07.718000+00:00,2021-03-03 23:30:08.505000+00:00,//compute.googleapis.com/projects/jinmmin-sand...,organizations/129624834409/sources/13627762962...,HIGH,NaN
2,Persistence: New User Agent,None,,2021-03-03 23:21:13.590000+00:00,2021-03-03 23:21:14.482000+00:00,//compute.googleapis.com/projects/jinmmin-sand...,organizations/129624834409/sources/13627762962...,HIGH,NaN
3,Persistence: New User Agent,None,,2021-03-03 23:34:37.697000+00:00,2021-03-03 23:34:39.037000+00:00,//compute.googleapis.com/projects/jinmmin-sand...,organizations/129624834409/sources/13627762962...,HIGH,NaN
4,Persistence: New User Agent,None,,2021-03-03 23:37:56.939000+00:00,2021-03-03 23:37:57.695000+00:00,//compute.googleapis.com/projects/jinmmin-sand...,organizations/129624834409/sources/13627762962...,HIGH,NaN
...,...,...,...,...,...,...,...,...,...
22200,resource_involved_in_coin_mining,None,gke-demo-green,2020-07-25 05:37:28.392000+00:00,2020-06-05 01:07:10.089000+00:00,//cloudresourcemanager.googleapis.com/projects...,organizations/129624834409/sources/76525934668...,SEVERITY_UNSPECIFIED,gke-demo-green
22201,resource_involved_in_coin_mining,None,csam-green-test-project-0,2020-07-25 05:40:11.059000+00:00,2020-05-26 23:31:38.078000+00:00,//cloudresourcemanager.googleapis.com/projects...,organizations/129624834409/sources/76525934668...,SEVERITY_UNSPECIFIED,csam-green-test-project-0
22202,resource_involved_in_coin_mining,None,gke-demo-green,2020-07-25 05:40:11.059000+00:00,2020-05-23 15:03:47.780000+00:00,//cloudresourcemanager.googleapis.com/projects...,organizations/129624834409/sources/76525934668...,SEVERITY_UNSPECIFIED,gke-demo-green
22203,resource_involved_in_coin_mining,None,gke-demo-green,2020-07-25 05:40:11.059000+00:00,2020-05-27 03:11:42.133000+00:00,//cloudresourcemanager.googleapis.com/projects...,organizations/129624834409/sources/76525934668...,SEVERITY_UNSPECIFIED,gke-demo-green
